# This notebook shows how to use Orchestrator APIs for user experiments

## Configure the Environment

In [1]:
import os
from getpass import getpass

os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

os.environ['FABRIC_BASTION_USERNAME']='pruth'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/id_rsa_fabric'

os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'
print('Please input private key passphrase. Press enter for no passphrase.')
os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV4'] = '192.168.11.226'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV6'] = '2600:2701:5000:a902::c'

## Setup the Experiment

#### Import FABRIC API

In [2]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib

## Create Slice
In Release 1.0, user is expected to assign the IP addresses manually. Please use the example comands indicated below:

## Configure Slice Parameters



In [3]:
slice_name = 'MySliceL2P2P'
site1 = 'TACC'
site2 = 'UTAH'
node1_name = 'Node1'
node2_name = 'Node2'
network_name='net1'
node1_nic_name = 'nic1'
node2_nic_name = 'nic2'
username = 'centos'
image = 'default_centos_8'
image_type = 'qcow2'
cores = 2
ram = 8
disk = 10

In [4]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=node1_name, site=site1)
    node1.set_capacities(cores=cores, ram=ram, disk=disk)
    node1.set_image(image)
    iface1 = node1.add_component(model='NIC_ConnectX_5', name=node1_nic_name).get_interfaces()[0]
    iface1.set_vlan(vlan='200')
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site2)
    node2.set_capacities(cores=cores, ram=ram, disk=disk)
    node2.set_image(image)
    iface2 = node2.add_component(model='NIC_ConnectX_5', name=node2_nic_name).get_interfaces()[0]
    iface2.set_vlan(vlan='200')
    
    # Network
    net1 = slice.add_l2network(name=network_name, interfaces=[iface1, iface2])

    #Submit Slice Request
    slice.submit(wait_progress=True)
except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()

Waiting for slice ................ Slice state: StableOK


## Get the Slice

In [5]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print(f"   Image             : {node.get_image()}")
        print(f"   Image Type        : {node.get_image_type()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print(f"   Components        :  ")
        for component in node.get_components():
            print(f"      Name             : {component.get_name()}")
            print(f"      Details          : {component.get_details()}")
            print(f"      Disk (G)         : {component.get_disk()}")
            print(f"      Units            : {component.get_unit()}")
            print(f"      PCI Address      : {component.get_pci_addr()}")
            print(f"      Model            : {component.get_model()}")
            print(f"      Type             : {component.get_type()}") 
        print(f"   Interfaces        :  ")
        for interface in node.get_interfaces():
            print(f"       Name                : {interface.get_name()}")
            print(f"           Bandwidth           : {interface.get_bandwidth()}")
            print(f"           VLAN                : {interface.get_vlan()}")            
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Node:
   Name              : Node1
   Cores             : 2
   RAM               : 8
   Disk              : 10
   Image             : default_centos_8
   Image Type        : qcow2
   Host              : tacc-w4.fabric-testbed.net
   Site              : TACC
   Management IP     : 129.114.110.113
   Reservation ID    : c283b295-3224-4976-a9d0-439f7cf7c5dc
   Reservation State : Active
   SSH Command       : ssh -i /Users/pruth/.ssh/id_rsa -J pruth@bastion-1.fabric-testbed.net centos@129.114.110.113
   Components        :  
      Name             : Node1-nic1
      Details          : Mellanox ConnectX-5 Dual Port 10/25GbE
      Disk (G)         : 0
      Units            : 1
      PCI Address      : ['0000:a1:00.0', '0000:a1:00.1']
      Model            : ConnectX-5
      Type             : SmartNIC
   Interfaces        :  
       Name                : Node1-nic1-p1
           Bandwidth           : 25
           VLAN                : 200
       Name                : Node1-nic1-p2
      

## Get the Nodes

Retrieve the node information and save the management IP address.


### Configure Node1

Use ssh to configure eth1 on  node 1.  

```
ip link add link eth1 name eth1.200 type vlan id 200
ip link set dev eth1.200 up
ip addr add 192.168.1.10/24 dev eth1.200

```

In [7]:
ip="192.168.1.10"
cidr="24"
os_iface='eth1'


try:
    node1 = slice.get_node(node1_name)
    vlan = node1.get_component(name=node1_nic_name).get_interfaces()[0].get_vlan()
    
    stdout, stderr = node1.execute(f'sudo ip link add link {os_iface} name {os_iface}.{vlan} type vlan id {vlan}')    
    stdout, stderr = node1.execute(f'sudo ip link set dev {os_iface}.{vlan} up')
    stdout, stderr = node1.execute(f'sudo ip addr add {ip}/{cidr} dev {os_iface}.{vlan}')
    stdout, stderr = node1.execute(f'sudo ip addr list {os_iface}.{vlan}')
    print (stdout)
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()


5: eth1.200@eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default qlen 1000
    link/ether b8:ce:f6:19:09:46 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.10/24 scope global eth1.200
       valid_lft forever preferred_lft forever
    inet6 fe80::bace:f6ff:fe19:946/64 scope link 
       valid_lft forever preferred_lft forever



### Configure Node2

Use ssh to configure eth1 on each Node 2.  

```
ip link add link eth1 name eth1.200 type vlan id 200
ip link set dev eth1.200 up
ip addr add 192.168.1.11/24 dev eth1.200
```

In [8]:
ip="192.168.1.11"
cidr="24"
os_iface='eth1'

try:
    node2 = slice.get_node(node2_name)
    vlan = node2.get_component(name=node2_nic_name).get_interfaces()[0].get_vlan()
    
    stdout, stderr = node2.execute(f'sudo ip link add link {os_iface} name {os_iface}.{vlan} type vlan id {vlan}')
    stdout, stderr = node2.execute(f'sudo ip link set dev {os_iface}.{vlan} up')
    stdout, stderr = node2.execute(f'sudo ip addr add {ip}/{cidr} dev {os_iface}.{vlan}')
    stdout, stderr = node2.execute(f'sudo ip addr list {os_iface}.{vlan}')
    print (stdout)
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

5: eth1.200@eth1: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default qlen 1000
    link/ether 04:3f:72:fa:75:88 brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.11/24 scope global eth1.200
       valid_lft forever preferred_lft forever
    inet6 fe80::63f:72ff:fefa:7588/64 scope link 
       valid_lft forever preferred_lft forever



## Delete Slice

In [9]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()